In [11]:
from fractions import Fraction

import numpy as np
import pandas as pd
from typing import Tuple

In [12]:
def display_table(constraint_coeffs, constraint_values, basis, step=None):
    """Função utilitária para exibir a tabela com a função objetivo e seu valor"""

    # Converte a matriz de coeficientes e os coeficientes objetivos para representação fracional
    frac_coeffs = [
        [Fraction(val).limit_denominator() for val in row] for row in constraint_coeffs
    ]

    # Converte os valores do RHS e o valor objetivo para representação fracional
    frac_values = [Fraction(val).limit_denominator() for val in constraint_values]

    # Determina o número de variáveis de decisão originais (excluindo variáveis de folga)
    num_decision_vars = len(constraint_coeffs[0]) - len(constraint_values)

    # Cria nomes de coluna para variáveis de decisão e variáveis de folga
    col_names = [f"x{i+1}" for i in range(num_decision_vars)]
    slack_names = [f"F{i+1}" for i in range(len(constraint_values))]
    col_names.extend(slack_names)

    tableau = pd.DataFrame(frac_coeffs, columns=col_names)
    tableau["RHS"] = frac_values
    tableau["Base"] = [
        f"x{b+1}" if b < num_decision_vars else f"F{b-num_decision_vars+1}"
        for b in basis
    ]

    # Reordena as colunas para colocar 'Basis' no lado esquerdo
    columns_order = ["Base"] + col_names + ["RHS"]
    tableau = tableau[columns_order]

    # print("\n" + ("=" * 50))
    # if step:
        # print(f"Passo: {step}")
    # print(tableau)
    # print("=" * 50 + "\n")


def simplex_method(
    objective_coeffs: np.ndarray,
    constraint_coeffs: np.ndarray,
    constraint_values: np.ndarray,
) -> Tuple[np.ndarray, float]:
    """
    Método Simplex para resolver um problema de programação linear na forma padrão com passos intermediários exibidos.
    """

    # Determina o número de restrições (linhas) e variáveis (colunas)
    num_constraints, num_variables = constraint_coeffs.shape
    num_slack_variables = num_constraints

    # Adiciona variáveis de folga para converter restrições para forma padrão
    constraint_coeffs = np.hstack([constraint_coeffs, np.eye(num_constraints)])
    objective_coeffs = np.hstack([objective_coeffs, np.zeros(num_slack_variables)])
    constraint_values = constraint_values.copy().astype(float)

    # Inicializa a base com as variáveis de folga
    basis = [i + num_variables for i in range(num_constraints)]

    display_table(constraint_coeffs, constraint_values, basis, step="Inicial")

    # Loop principal do método simplex
    iteration = 0
    while True:
        iteration += 1
        # Calcula o custo das variáveis básicas
        basic_cost = np.array([objective_coeffs[j] for j in basis])

        # Calcula a matriz de custo reduzido
        reduced_cost_matrix = (
            basic_cost @ np.linalg.inv(constraint_coeffs[:, basis]) @ constraint_coeffs
        )

        # Calcula a diferença entre os coeficientes da função objetivo e a matriz de custo reduzido
        diff_cost = objective_coeffs - reduced_cost_matrix

        # Se todos os valores em diff_cost forem não positivos, a solução atual é ótima
        if np.all(diff_cost <= 0):
            break

        # Determina a variável que entra (valor positivo mais alto em diff_cost)
        entering_variable = np.argmax(diff_cost).astype(int)

        # Calcula as razões para determinar a variável que sai
        ratios = np.array(
            [
                constraint_values[i] / constraint_coeffs[i, entering_variable]
                if constraint_coeffs[i, entering_variable] > 0
                else np.inf
                for i in range(num_constraints)
            ]
        )

        exiting_variable_idx = np.argmin(ratios)
        basis[exiting_variable_idx] = entering_variable

        # Pivô em torno da variável que entra e sai
        constraint_values[exiting_variable_idx] /= constraint_coeffs[
            exiting_variable_idx, entering_variable
        ]
        constraint_coeffs[exiting_variable_idx, :] /= constraint_coeffs[
            exiting_variable_idx, entering_variable
        ]

        for i in range(num_constraints):
            if i != exiting_variable_idx:
                factor = -constraint_coeffs[i, entering_variable]
                constraint_coeffs[i, :] += (
                    factor * constraint_coeffs[exiting_variable_idx, :]
                )
                constraint_values[i] += factor * constraint_values[exiting_variable_idx]

        display_table(
            constraint_coeffs, constraint_values, basis, step=f"Iteração {iteration}"
        )

        # display_table(constraint_coeffs, constraint_values, basis, step=f"Iteração {iteration}")

    # Calcula a solução
    solution = np.zeros(num_variables + num_slack_variables)
    for i, b in enumerate(basis):
        solution[b] = constraint_values[i]
    objective_value = objective_coeffs @ solution

    return solution, objective_value

In [13]:
from scipy.optimize import linprog


def solve_simplex(c: np.ndarray, A: np.ndarray, b: np.ndarray):
    solution, value = simplex_method(c, A, b)
    print(f"O - Solution: {solution}, Optimal value: {value}")


def solve_scipy(c: np.ndarray, A: np.ndarray, b: np.ndarray):
    # Note: linprog tries to minimize, not maximize, so we negate c
    bounds = [(0, None), (0, None)]

    result = linprog(-c, A_ub=A, b_ub=b, bounds=bounds)

    result_x = result.x
    result_fun = -result.fun  # We negate to get the maximized value
    result_s = result.slack
    result_concat = np.concatenate([result_x, result_s])
    print(f"S - Solution: {result_concat}, Optimal value: {result_fun}")

In [14]:
# Example problem
# Maximize z = 3x1 + 2x2
# Subject to:
# 2x1 + x2 <= 8
# x1 + 2x2 <= 6
# x1, x2 >= 0

# c is the coefficients of the objective function
# A is the matrix of coefficients of the constraints
# b is the vector of values on the right side of the constraints
z_coeffs = np.array([5, 3.0])
constraints_coeff = np.array([[2, 3.0], [2, 1], [1, -1]])
constrains_values = np.array([15, 9.0, 3])

solve_simplex(z_coeffs, constraints_coeff, constrains_values)
solve_scipy(z_coeffs, constraints_coeff, constrains_values)

O - Solution: [3. 3. 0. 0. 3.], Optimal value: 24.0
S - Solution: [3. 3. 0. 0. 3.], Optimal value: 24.0


In [15]:
z_coeffs = np.array([3, 2.0])
constraints_coeff = np.array([[2, 4.0], [-1, 4], [2, -1], [1, -3]])
constrains_values = np.array([22, 10, 7, 1.0])

solve_simplex(z_coeffs, constraints_coeff, constrains_values)
solve_scipy(z_coeffs, constraints_coeff, constrains_values)

O - Solution: [5. 3. 0. 3. 0. 5.], Optimal value: 21.000000000000004
S - Solution: [5. 3. 0. 3. 0. 5.], Optimal value: 21.0
